
We're about to create and use our own MCP Server and MCP Client!

It's pretty simple, but it's not super-simple. The excitment around MCP is about how easy it is to share and use other MCP Servers - making our own does involve a bit of work.

Let's review some python code made mostly by a hard-working Engineering Team:

accounts.py

In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

True

In [2]:
from accounts import Account

In [3]:
account = Account.get("Dhanu")
account

Account(name='dhanu', balance=10000.0, strategy='', holdings={}, transactions=[], portfolio_value_time_series=[])

In [4]:
account.buy_shares("AMZN", 3, "Because this bookstore website looks promising")

'Completed. Latest details:\n{"name": "dhanu", "balance": 9738.478, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 87.174, "timestamp": "2025-10-24 16:54:07", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:54:07", 9810.478]], "total_portfolio_value": 9810.478, "total_profit_loss": -189.52200000000084}'

In [5]:
account.report()

'{"name": "dhanu", "balance": 9738.478, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 87.174, "timestamp": "2025-10-24 16:54:07", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:54:07", 9810.478], ["2025-10-24 16:54:13", 10026.478]], "total_portfolio_value": 10026.478, "total_profit_loss": 26.477999999999156}'

In [6]:
account.list_transactions()

[{'symbol': 'AMZN',
  'quantity': 3,
  'price': 87.174,
  'timestamp': '2025-10-24 16:54:07',
  'rationale': 'Because this bookstore website looks promising'}]

### Now we write an MCP server and use it directly!

In [8]:
# Now let's use our accounts server as an MCP server

params = {"command": "uv", "args": ["run", "accounts_server.py"]}
async with MCPServerStdio(params=params, client_session_timeout_seconds=60) as server:
    mcp_tools = await server.list_tools()


In [9]:
mcp_tools

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta=None),
 Tool(name='buy_shares', title=None, description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The ration

In [10]:
instructions = "You are able to manage an account for a client, and answer questions about the account."
request = "My name is Dhanu and my account is under the name Dhanu. What's my balance and my holdings?"
model = "gpt-4.1-mini"

In [11]:

async with MCPServerStdio(params=params, client_session_timeout_seconds=30) as mcp_server:
    agent = Agent(name="account_manager", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("account_manager"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))


Dhanu, your current account balance is $9,738.48. As for your holdings, you have 3 shares of AMZN (Amazon). Is there anything specific you'd like to do with your account?

### Now let's build our own MCP Client

In [12]:
from accounts_client import get_accounts_tools_openai, read_accounts_resource, list_accounts_tools

mcp_tools = await list_accounts_tools()
print(mcp_tools)
openai_tools = await get_accounts_tools_openai()
print(openai_tools)

[Tool(name='get_balance', title=None, description='Get the cash balance of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_balanceArguments', 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta=None), Tool(name='get_holdings', title=None, description='Get the holdings of the given account name.\n\n    Args:\n        name: The name of the account holder\n    ', inputSchema={'properties': {'name': {'title': 'Name', 'type': 'string'}}, 'required': ['name'], 'title': 'get_holdingsArguments', 'type': 'object'}, outputSchema=None, icons=None, annotations=None, meta=None), Tool(name='buy_shares', title=None, description="Buy shares of a stock.\n\n    Args:\n        name: The name of the account holder\n        symbol: The symbol of the stock\n        quantity: The quantity of shares to buy\n        rationale: The rational

In [13]:
request = "My name is Dhanu and my account is under the name Dhanu. What's my balance?"

with trace("account_mcp_client"):
    agent = Agent(name="account_manager", instructions=instructions, model=model, tools=openai_tools)
    result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

Dhanu, your current account balance is $9,738.48. Is there anything specific you would like to do with your account?

In [14]:
context = await read_accounts_resource("Dhanu")
print(context)

{"name": "dhanu", "balance": 9738.478, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 87.174, "timestamp": "2025-10-24 16:54:07", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:54:07", 9810.478], ["2025-10-24 16:54:13", 10026.478], ["2025-10-24 16:59:50", 9744.478]], "total_portfolio_value": 9744.478, "total_profit_loss": -255.52200000000084}


In [15]:
from accounts import Account
Account.get("Dhanu").report()

'{"name": "dhanu", "balance": 9738.478, "strategy": "", "holdings": {"AMZN": 3}, "transactions": [{"symbol": "AMZN", "quantity": 3, "price": 87.174, "timestamp": "2025-10-24 16:54:07", "rationale": "Because this bookstore website looks promising"}], "portfolio_value_time_series": [["2025-10-24 16:54:07", 9810.478], ["2025-10-24 16:54:13", 10026.478], ["2025-10-24 16:59:50", 9744.478], ["2025-10-24 16:59:52", 9828.478]], "total_portfolio_value": 9828.478, "total_profit_loss": -171.52200000000084}'